In [4]:
import os
import librosa
import time
import glob
import csv
import cv2
import librosa.display
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers.normalization import BatchNormalization
from numpy import array, asarray, ndarray

### Transforms images from a directory in spectrograms

In [ ]:
def audio_to_image(filepath, savepath='./images/'):
    
    filepath = os.path.abspath(filepath)
    savepath = os.path.abspath(savepath)
    mpl.rcParams['savefig.pad_inches'] = 0
    # Reads every audio file from filepath, returns amplitude in time(y) and sample rate(sr)
    for folder in os.listdir(filepath):
        folderpath = '{}/{}'.format(filepath, folder)
        
        for filename in os.listdir(folderpath):
            fullpath = '{}/{}'.format(folderpath, filename)
            print(fullpath)
            audio_wave, sample_rate = librosa.load(fullpath)

            # Applies Fourier transform in audio's amplitude
            fourier = librosa.stft(audio_wave)

            # Converts amplitude to DBs
            """
            ref:scalar or callable
                If scalar, the amplitude abs(S) is scaled relative to ref: 20 * log10(S / ref). 
                Zeros in the output correspond to positions where S == ref.

                If callable, the reference value is computed as ref(S).
            """
            D = librosa.amplitude_to_db(np.abs(fourier), ref=np.max)
            
           
            librosa.display.specshow(D, x_axis='time', y_axis='log')
            plt.axis('off')
            #plt.axis('off')
            
            savefolder = '{}/{}'.format(savepath, folder)
            
            if not os.path.exists(savefolder):
                os.makedirs(savefolder)
            
            plt.savefig(savefolder + '/' + filename.split('.')[0] + '-spectogram.png', bbox_inches='tight',  pad_inches=0)

In [ ]:
audio_to_image('./teste/')

In [63]:
### Loads audio Data CSV
df_tracks = pd.read_csv('fma_metadata/tracks.csv')
df_tracks.dropna(subset = ['genre_top'], inplace = True)

In [123]:
# Training options

# Number of samples propagated through the network
batch_size = 25
epochs = 1
training_size = 0.7

# Image's dimensions
img_rows, img_cols = 224, 224


x_test = []
x_train = []

y_test = []
y_train = []
tempY = []

# Training data size
length = int(95*training_size)

In [131]:
def create_set(filepath):
    
    x_test = []
    x_train = []

    y_test = []
    y_train = []
    tempY = []
    
    filepath = os.path.abspath(filepath)
    
    i = 0
    for folder in os.listdir(filepath):
        folderpath = '{}/{}'.format(filepath, folder)
        
        for filename in os.listdir(folderpath):
            fullpath = '{}/{}'.format(folderpath, filename)
            
            current_track_id = filename.split('-')[0]
            current_track_id = int(current_track_id)
            
            genre = df_tracks[df_tracks.track_id == current_track_id]
            genre = genre.genre_top.to_list()[0]
            print(i, fullpath, current_track_id, genre)
                  
            img = cv2.imread(fullpath)
            
            if(i < length):
                x_train.append(img)
                y_train.append(genre)
            else:
                x_test.append(img)
                y_test.append(genre)
        
        i = i + 1

    return x_train, y_train, x_test, y_test   

In [132]:
x_train, y_train, x_test, y_test = create_set('images')

# Converting the data from lists to numpy arrays

x_train = asarray(x_train, dtype = float)
x_test = asarray(x_test, dtype = str)
y_train = asarray(y_train, dtype = float)
y_test = asarray(y_test, dtype = str)

# Scaling down the RGB data
x_train /= 255
x_test /= 255

# Printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

0 /home/iago/Documentos/visao_computacional/music-genre-ai/images/001/001069-spectogram.png 1069 Experimental
0 /home/iago/Documentos/visao_computacional/music-genre-ai/images/001/001040-spectogram.png 1040 Rock
0 /home/iago/Documentos/visao_computacional/music-genre-ai/images/001/001039-spectogram.png 1039 Rock
0 /home/iago/Documentos/visao_computacional/music-genre-ai/images/001/001066-spectogram.png 1066 Experimental
1 /home/iago/Documentos/visao_computacional/music-genre-ai/images/033/033049-spectogram.png 33049 Folk
1 /home/iago/Documentos/visao_computacional/music-genre-ai/images/033/033064-spectogram.png 33064 Electronic
1 /home/iago/Documentos/visao_computacional/music-genre-ai/images/033/033069-spectogram.png 33069 International
1 /home/iago/Documentos/visao_computacional/music-genre-ai/images/033/033070-spectogram.png 33070 International
1 /home/iago/Documentos/visao_computacional/music-genre-ai/images/033/033050-spectogram.png 33050 Folk
1 /home/iago/Documentos/visao_computa

ValueError: setting an array element with a sequence.

In [ ]:
#defining the model
model = Sequential()
#input
model.add(Conv2D(128,data_format = 'channels_last', kernel_size=(3, 3),
                 input_shape=(img_rows, img_cols,3)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
#convolutions

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

#dense layers
model.add(Flatten())

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dropout(0.15))

#output
model.add(Dense(1))

#printing model summary
print(model.summary())

#compiling the model
model.compile(optimizer='RMSprop', loss='mse', metrics=['mae'])

#training the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

#testing the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [117]:
teste = int("13220")
df_tracks[df_tracks.track_id == teste]

,track_id,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,...,publisher,tags,title,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
7598,13220,small,320000,0,NaN,2009-05-19 10:57:27,NaN,192,1,Pop,...,2,NaN,[],Horse On A Rock,NaN,NaN,NaN,NaN,NaN,NaN
